In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [ ]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [ ]:
# Saca links de páginas

links = ['http://www.faperj.br/?id=3532.3.5', 'http://www.faperj.br/?id=3504.3.7', 'http://www.faperj.br/?id=3480.3.9', 'http://www.faperj.br/?id=3382.3.2', 'http://www.faperj.br/?id=3234.3.4', 'http://www.faperj.br/?id=1132.3.0', 'http://www.faperj.br/?id=3031.3.3', 'http://www.faperj.br/?id=2456.3.4', 'http://www.faperj.br/interna.phtml?obj_id=8768', 'http://www.faperj.br/interna.phtml?obj_id=7935', 'http://www.faperj.br/interna.phtml?obj_id=7019', 'http://www.faperj.br/interna.phtml?obj_id=5178', 'http://www.faperj.br/redir.php?obj_id=4340', 'http://www.faperj.br/redir.php?obj_id=3835', 'http://www.faperj.br/redir.php?obj_id=92']

anos = ['2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2002 a 2006']

In [ ]:
pdf_proyectos = []
titulo_proyectos = []
anos_proyectos = []
descripciones_proyectos = []

y = 1
n = 0
for link in links:

    parser_pag = parser(link)
    texto_bruto = parser_pag.xpath('//p//a//text()')
    ### TITULO

    titulo_base = []

    for t in texto_bruto:

        if ' – ' in t or t.startswith('Edital') or t.startswith('Programa') or t.startswith('Apoio') or t.startswith('Resultado:'):
            if len(t) != 24 and 'Programme' not in t:
                titulo_base.append(t)

    ######################## PDFS

    base = 'http://www.faperj.br/'
        
    titulo = []
    pdfs = []
    descripciones = []
    base = 'http://www.faperj.br/'
    
    
    for i in range(1, len(titulo_base)+10):
        pdf_brutos_base = parser_pag.xpath('//div[@id="texto"]//p['+str(i)+']/strong//a//@href')
            
        #PDFS
        pdf = ''
        if len(pdf_brutos_base) != 0:
            for p in pdf_brutos_base:
                if p.endswith('.pdf') and p.startswith(base) == False:
                    pdf = pdf + base + p + ', '
                elif p.endswith('.pdf') and p.startswith(base):
                    pdf = pdf+ p + ', '
                else:
                    pdf = ''
    
        pdfs.append(pdf.strip(', '))

        try:

            titulo_brutos_base = parser_pag.xpath('//div[@id="texto"]//p['+str(i)+']/strong//a/text()')
            titulo.append(titulo_brutos_base[0])
        except:

            #titulo_brutos_base = parser_pag.xpath('//div[@id="texto"]//p[@class="MsoNormal"]['+str(i)+']//a//text()')
            #titulo.append(titulo_brutos_base)
            continue


    ################################
    
    anos_pro = [anos[n] for i in range(len(titulo))]
    
    if len(pdfs) == 0:
        pdfs = [pdfs.append('') for i in range(len(titulo))]
    n+=1

    print(link)
    #print('pdfs:', len(pdfs[0:len(titulo)]))
    print('Titulos:', len(titulo))
    #print('Anos:', len(anos_pro))
    #print('-------------')
    
    pdf_proyectos += pdfs[0:len(titulo)]
    titulo_proyectos += titulo
    anos_proyectos += anos_pro
 

In [58]:
links = ['http://www.faperj.br/?id=3031.3.3', 'http://www.faperj.br/?id=2456.3.4', 'http://www.faperj.br/interna.phtml?obj_id=8768', 'http://www.faperj.br/interna.phtml?obj_id=7935', 'http://www.faperj.br/interna.phtml?obj_id=7019', 'http://www.faperj.br/interna.phtml?obj_id=5178', 'http://www.faperj.br/redir.php?obj_id=4340', 'http://www.faperj.br/redir.php?obj_id=3835', 'http://www.faperj.br/redir.php?obj_id=92']

parser_pag = parser(links[0])

links = parser_pag.xpath('//div[@id="texto"]//p//a[1]//@href[1]')
titulos = parser_pag.xpath('//div[@id="texto"]//p//a[1]//text()[1]')

In [62]:
links[100]
#titulos[100]

'http://www.faperj.br/interna.phtml?obj_id=8768'

In [ ]:
faperj = pd.DataFrame()

faperj['Titulo'] = titulo_proyectos
faperj['Ano'] = anos_proyectos
faperj['PDFs'] = pdf_proyectos
faperj

In [ ]:
faperj
#faperj.to_excel('brasil_faprej.xlsx')